此次练习中，我们使用[Human Activity Recognition Using Smartphones](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)数据集。它通过对参加测试者的智能手机上安装里一个传感器而采集了参加测试者每天的日常活动（ADL）。目标是将日常活动分成六类（walking, walking upstairs, walking downstairs, sitting, standing, and laying）。

该数据集也可以在Kaggle网站上获得：https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones/downloads/human-activity-recognition-with-smartphones.zip 

把训练文件重新命名为 Human_Activity_Recognition_Using_Smartphones_Data.csv 

## 第一步：导入数据

* 查看数据类型---因为有太多的列，所以最好使用value_counts
* 判断其中的小数数值是否需要尺度缩放
* 检查数据中各活动类型的划分
* 把活动类型标签编码成一个整数

In [51]:
import pandas as pd
import numpy as np

filepath = 'Human_Activity_Recognition_Using_Smartphones_Data.csv'
data = pd.read_csv(filepath)

所有列的数据类型都是浮点数，除了活动标签列。

In [52]:
data.dtypes.value_counts()

float64    561
int64        1
object       1
dtype: int64

In [53]:
data.dtypes.tail()

angle(X,gravityMean)    float64
angle(Y,gravityMean)    float64
angle(Z,gravityMean)    float64
subject                   int64
Activity                 object
dtype: object

数据都已经全部被缩放到-1到1之间了。

In [54]:
data.iloc[:, :-1].min().value_counts()

-1.000000    466
-1.000000      2
-0.999893      2
-0.999996      2
-0.995377      2
-0.963099      1
-0.999998      1
-0.899914      1
-0.954173      1
-0.773597      1
-0.931208      1
-0.493874      1
-0.966591      1
-1.000000      1
-0.947304      1
-0.999873      1
-1.000000      1
-0.999960      1
-0.999658      1
-0.905918      1
-0.999162      1
-0.799069      1
-0.954486      1
-0.999994      1
-0.966345      1
-0.953894      1
-0.999994      1
-0.999999      1
 1.000000      1
-0.999983      1
            ... 
-0.617636      1
-0.999849      1
-0.999997      1
-0.999857      1
-0.895269      1
-0.997664      1
-0.999152      1
-0.568157      1
-0.914050      1
-0.753754      1
-0.995357      1
-0.925897      1
-0.759790      1
-0.998014      1
-0.908981      1
-0.999967      1
-0.999999      1
-0.999999      1
-0.830559      1
-1.000000      1
-0.999889      1
-0.999989      1
-0.535222      1
-0.554000      1
-1.000000      1
-0.999835      1
-1.000000      1
-0.976580     

In [55]:
data.iloc[:, :-1].max().value_counts()

1.000000    452
0.994731      2
0.968307      1
0.974333      1
0.465197      1
0.624373      1
0.928416      1
0.968254      1
0.945207      1
0.992636      1
0.894495      1
0.978449      1
0.991592      1
0.919662      1
0.975550      1
0.880335      1
0.881452      1
0.925207      1
0.836860      1
0.807078      1
0.996078      1
0.999015      1
0.920890      1
0.857113      1
0.914147      1
0.794498      1
0.996585      1
0.821090      1
0.969311      1
0.999269      1
           ... 
0.933547      1
0.897898      1
0.956845      1
0.806452      1
0.993741      1
0.960000      1
0.967664      1
0.923077      1
0.904480      1
0.985138      1
0.930317      1
0.908361      1
0.914892      1
0.678628      1
0.641321      1
0.923377      1
0.805064      1
0.660042      1
0.975855      1
0.964576      1
0.632415      1
0.997207      1
0.807375      1
0.903571      1
0.993524      1
0.949350      1
0.846610      1
0.836728      1
0.914345      1
0.738953      1
Length: 110, dtype: int6

检查数据中各活动类型的划分---已经比较平衡了。

In [56]:
data.Activity.value_counts()

LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: Activity, dtype: int64

Scikit learn的分类器不接受一个稀疏矩阵作为预测列。所以，可以使用`LabelEncoder`将活动标签编码为整数。

In [57]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Activity'] = le.fit_transform(data.Activity)
data['Activity'].sample(5)

4894    2
1635    2
983     5
4757    1
2267    1
Name: Activity, dtype: int64

## 第二步：划分训练数据和测试数据

* 可以考虑使用Scikit-learn中的`StratifiedShuffleSplit`，以保证划分后的数据集中每个类别个案的比例与整个数据集相同。


In [58]:
feature_cols = data.columns[:-1]

from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1,test_size=0.3, random_state=42)

train_idx, test_idx = next(strat_shuf_split.split(data[feature_cols], data.Activity))

# Create the dataframes
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'Activity']

X_test  = data.loc[test_idx, feature_cols]
y_test  = data.loc[test_idx, 'Activity']

In [59]:
y_train.value_counts(normalize=True)

0    0.191411
2    0.186941
1    0.174893
3    0.166731
5    0.145939
4    0.134085
Name: Activity, dtype: float64

In [60]:
y_test.value_counts(normalize=True)

0    0.191296
2    0.186763
1    0.174977
3    0.166818
5    0.145966
4    0.134180
Name: Activity, dtype: float64

## 第三步：训练模型

* 用所有特征训练一个基本的使用缺省参数的逻辑回归模型。
* 分别用L1和L2正则化来训练一个模型，使用交叉验证确定超参数的值。注意，正则化模型，尤其是L1模型可能需要一定训练时间。

In [63]:
# 请在此处填写你的代码（训练一个基本的使用缺省参数的逻辑回归模型）


In [62]:
from sklearn.linear_model import LogisticRegressionCV

# L1 正则化的逻辑回归
lr_l1 = LogisticRegressionCV(Cs=10, cv=4, penalty='l1', solver='liblinear').fit(X_train, y_train)

In [32]:
# 请在此处填写你的代码（L2 正则化的逻辑回归）


## 第四步：

* 输出上面训练出的三个模型中每个特征的系数；
* 并绘制成图来比较它们的差异 （每个类别一张图）

In [33]:
# 请在此处填写你的代码（输出各模型训练到的特征系数值）


In [34]:
# 请在此处填写你的代码（绘制6张图）


## 第五步：预测数据

* 将每个模型预测的类别和概率值都保存下来。

In [35]:
# 请在此处填写你的代码


## 第六步：评价模型

对每个模型，分别计算下面的各评测指标值： 

* accuracy
* precision
* recall
* fscore
* confusion matrix

In [30]:
# 请在此处填写你的代码
